In [116]:
%pip install --upgrade --quiet  langchain-google-genai
%pip install python-dotenv --quiet
%pip install --upgrade --quiet  langchain pandas fpdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [117]:
import os
import sys
import json
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import pandas as pd
from fpdf import FPDF
load_dotenv()

True

In [118]:
KEY = os.getenv("GOOGLE_API_KEY")

In [119]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=KEY,temperature=.7)


In [120]:
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros:**

* **Simplicity and readability:** Python's syntax is designed to be easy to read and write, making it accessible to beginners and experienced developers alike.
* **Extensive library support:** Python has a vast collection of libraries and frameworks, covering a wide range of domains, such as data science, machine learning, web development, and automation.
* **Cross-platform compatibility:** Python can run on multiple operating systems, including Windows, macOS, and Linux, making it a portable option for development.
* **Interpreted nature:** Python is an interpreted language, which means that it allows for rapid development and testing without the need for compilation.
* **Strong community support:** Python has a large and active community, providing resources, documentation, and support for developers.

**Cons:**

* **Speed:** Python can be slower than compiled languages, such as C++ or Java, especially for computationally intensive tasks.
* **Memory management:** Python us

***design of response***

In [121]:
RESPONSE_JSON = {
    "1":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "2":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "3":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "4":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
}

In [122]:
TEMPLATE_1='''
   TEXT: {text}
You are an expert in creating multiple choice questions. Given the above text, it is your job to create a quiz of {number} questions for {subject} students in a {tone} tone. Ensure the questions are unique and relevant to the text.
Use the format provided below as a guide to construct your quiz, making sure to include {number} MCQs:

{response_json}

'''

In [123]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE_1
)

In [124]:
quiz_chain = LLMChain(
    llm=llm,
    prompt = quiz_generation_prompt,
    output_key = "quiz",
    verbose=True
)

In [125]:
TEMPLATE_2 = '''
    You are expert in english gramer and writer. Give a muitiple choice for students in subject {subject}. \
    You need to evaluate the complexity of question and give a complete ananylsis of quiz. Only use at max 50 words for each.\
    Quiz_MCQs:
    {quiz}

    check from expert English writer of the above quiz:
'''

In [126]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE_2
)

In [127]:
review_chain = LLMChain(
    llm=llm,
    prompt = quiz_evaluation_prompt,
    output_key = "review",
    verbose=True
)

In [128]:
generate_evalution_chain = SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [129]:
file_path = "/Users/naveen/Developer/GenAI/quiz_generator/data.txt"

In [130]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [131]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "2": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "3": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "4": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}}'

In [132]:
NUMBER = 10
SUBJECT = "deep Learning",
TONE = "simple"

In [133]:
res = generate_evalution_chain.__call__(
        {
        "text":TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

   TEXT: Collaborative filtering (CF) is a successful approach commonly used by many recommender systems.
Conventional CF-based methods use the ratings given to items by users as the sole source of information for learning to make recommendation. 
However, the ratings are often very sparse in many applications, causing CF-based methods to degrade significantly in their recommendation performance. 
To address this sparsity problem, auxiliary information such as item content information may be utilized. Collaborative topic regression (CTR) is an appealing recent method taking this approach which tightly couples the two components that learn from two different sources of information. 
Nevertheless, the latent representation learned by CTR may not be very effective when the auxiliary information is very sparse. 
To address this problem, we generalize recently advances in deep learning fro

In [134]:
res

{'text': 'Collaborative filtering (CF) is a successful approach commonly used by many recommender systems.\nConventional CF-based methods use the ratings given to items by users as the sole source of information for learning to make recommendation. \nHowever, the ratings are often very sparse in many applications, causing CF-based methods to degrade significantly in their recommendation performance. \nTo address this sparsity problem, auxiliary information such as item content information may be utilized. Collaborative topic regression (CTR) is an appealing recent method taking this approach which tightly couples the two components that learn from two different sources of information. \nNevertheless, the latent representation learned by CTR may not be very effective when the auxiliary information is very sparse. \nTo address this problem, we generalize recently advances in deep learning from i.i.d. input to non-i.i.d. (CF-based) input and propose in this paper a hierarchical Bayesian m

In [135]:
quiz = res.get('quiz')

In [136]:
quiz = json.loads(quiz)
type(quiz)
data = quiz

In [137]:
quiz_data = []
for key,value in quiz.items():
    mcq = value.get('mcq')
    options = [f'{key}: {val}' for key, val in value.get('options').items()]
    correct = value.get('correct')
    quiz_data.append({'MCQ':mcq,"options":options, "Correct Answer": correct})

In [138]:
quiz_data

[{'MCQ': 'What is a common approach used by many recommender systems?',
  'options': ['a: Collaborative filtering',
   'b: Content-based filtering',
   'c: Hybrid filtering',
   'd: Rule-based filtering'],
  'Correct Answer': 'a'},
 {'MCQ': 'What is the main problem with conventional CF-based methods?',
  'options': ['a: Data sparsity',
   'b: Cold start',
   'c: Overfitting',
   'd: Scalability'],
  'Correct Answer': 'a'},
 {'MCQ': 'What additional information can be used to address the sparsity problem in CF-based methods?',
  'options': ['a: Item content information',
   'b: User demographics',
   'c: Social network data',
   'd: All of the above'],
  'Correct Answer': 'd'},
 {'MCQ': 'What is the name of the method that tightly couples learning from ratings and content information?',
  'options': ['a: Collaborative topic regression (CTR)',
   'b: Matrix factorization',
   'c: Neighborhood-based methods',
   'd: Deep autoencoders'],
  'Correct Answer': 'a'},
 {'MCQ': 'What is the mai

In [139]:
pd.DataFrame(quiz_data)

,MCQ,options,Correct Answer
0,What is a common approach used by many recomme...,"[a: Collaborative filtering, b: Content-based ...",a
1,What is the main problem with conventional CF-...,"[a: Data sparsity, b: Cold start, c: Overfitti...",a
2,What additional information can be used to add...,"[a: Item content information, b: User demograp...",d
3,What is the name of the method that tightly co...,"[a: Collaborative topic regression (CTR), b: M...",a
4,What is the main limitation of the latent repr...,[a: It may not be effective when auxiliary inf...,a
5,What is the main idea behind collaborative dee...,[a: Combining deep representation learning and...,a
6,What type of input data does CDL handle?,"[a: i.i.d. input, b: Non-i.i.d. input, c: Stru...",b
7,What is the main benefit of CDL over existing ...,[a: It can significantly advance the state of ...,a
8,In what domains have CDL experiments been cond...,"[a: E-commerce, b: Movie recommendation, c: Mu...",d
9,What is the main purpose of deep representatio...,[a: To extract meaningful features from conten...,a


In [140]:
df = pd.DataFrame(quiz_data)

In [141]:
class MCQPDF(FPDF):
    def __init__(self, include_answers=True):
        super().__init__()
        self.include_answers = include_answers
        self.title_added = False
        self.add_page()
        self.set_auto_page_break(auto=True, margin=30)

    def header(self):
        title = 'MCQ Assessment - Answers Included' if self.include_answers else 'MCQ Assessment - For Students'
        if not self.title_added:
            self.set_font('Arial', 'I', 16)
            self.cell(0, 10, title, 0, 1, 'C')
            self.cell(0, 10, '', 0, 1, 'C')
            self.title_added = True

    def footer(self):

        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, 'Generated by Hana-AI', 0, 0, 'L')
        self.cell(0, 10, 'Page %s' % self.page_no(), 0, 0, 'C')

    def create_document(self):
        self.set_font("Arial", size=12)
        for key, value in data.items():
            self.set_font("Arial", 'B', 12)
            self.multi_cell(0, 10, f"Question {key}: {value['mcq']}")
            self.set_font("Arial", size=12)
            for opt_key, opt_value in value['options'].items():
                self.multi_cell(0, 10, f"{opt_key}: {opt_value}")
            if self.include_answers:
                self.multi_cell(0, 10, f"Correct Answer: {value['correct']}")
            self.ln(10)


In [142]:
pdf_with_answers = MCQPDF(include_answers=True)
pdf_with_answers.set_author("Hana-AI")
pdf_with_answers.create_document()
pdf_with_answers.output("MCQs_by_HanaAi_with_Answers.pdf")

pdf_without_answers = MCQPDF(include_answers=False)
pdf_without_answers.set_author("Hana-AI")
pdf_without_answers.create_document()
pdf_without_answers.output("MCQs_by_HanaAi_.pdf")

''

In [143]:
print("fucked successfully....")

fucked successfully....
